# 非階層的クラスタリング

クラスタ数は推定結果に合わせて調整してください。

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# フィードデータの読み込み
feeds = pd.read_csv('data/output_en.csv')

# title と summary を結合して text 列を作成
feeds['text'] = feeds['title'].str.cat(feeds['summary'], sep='. ', na_rep='')

# 不要になった列を削除した処理用の DataFrame
df = feeds.drop(['title', 'summary'], axis=1)

# 確認
df

### 英語テキストに対する前処理

以下をまとめて行う関数 preprocess() を定義
- トークン化（単語に分割）
- 小文字化
- ストップワードの除去
- ステミング
- 見出し語化

In [ ]:
symbols_to_remove = r'["`,.' + r"'" + r']'
stop_words = nltk.corpus.stopwords.words('english')
stop_words += ["'", '"', ':', ';', '.', ',', '-', '!', '?', "'s", '`', '•', '%']
stop_words += ['–', '—', '‘', '’', '“', '”', '…', '|', '#', '$', '&', "''", '(', ')']
stemmer = nltk.stem.porter.PorterStemmer()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# 品詞の名称を変換
def wordnet_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    return None

def preprocess(text):
    tokens = []
    # 品詞のタグ付けをした各トークンについて
    for t in nltk.pos_tag(nltk.tokenize.word_tokenize(text.replace('-', ' '))):
        # 小文字化
        t0 = t[0].lower()
        # 不要な文字の削除
        t0 = re.sub(symbols_to_remove, '', t0)
        # 空文字列になったら次へ
        if t0 == '':
            continue
        # stop_words に含まれていないトークンのみを残す
        if t0 in stop_words:
            continue
        # カンマ区切りが入った数値からカンマを削除
        if t[1] == 'CD':
            t0 = t0.replace(',', '')
        # 見出し語化
        tag = wordnet_tag(t[1])
        if tag is None:
            t0 = lemmatizer.lemmatize(t0)
        else:
            t0 = lemmatizer.lemmatize(t0, tag)
        # ステミング
        t0 = stemmer.stem(t0)
        # リストに追加
        tokens.append(t0)
    # トークンのリストを返す
    return tokens

### テキストのベクトル化

- TF-IDF

In [ ]:
# TfidfVectorizer によりベクトル化
vectorizer = TfidfVectorizer(tokenizer=preprocess)
vector = vectorizer.fit_transform(df.text)

### クラスタ数の推定

- 非階層的クラスタ分析ではクラスタ数を決めてデータをグループに分割
 - エルボー法
 - シルエット分析

In [ ]:
# エルボー法
# https://github.com/rasbt/python-machine-learning-book-2nd-edition/blob/master/code/ch11/ch11.py
x_range = range(1, 25)
distortions = []
for n in x_range:
    model = KMeans(n_clusters=n, random_state=0)
    model.fit(vector)
    distortions.append(model.inertia_)

sns.lineplot(x=x_range, y=distortions, marker='o')
plt.xlabel('Number of Clusters')
plt.show()

In [ ]:
# シルエット分析
# https://github.com/rasbt/python-machine-learning-book-2nd-edition/blob/master/code/ch11/ch11.py
from sklearn.metrics import silhouette_samples
from matplotlib import cm

def silhouette(X, n, plot=True):
    model = KMeans(n_clusters=n, random_state=0)
    model.fit(X)
    cluster_labels = set(model.labels_)
    n_clusters = len(cluster_labels)
    silhouette_vals = silhouette_samples(X, model.labels_, metric='euclidean')
    if plot:
        y_lower, y_upper = 0, 0
        yticks = []
        for i, c in enumerate(cluster_labels):
            c_silhouette_vals = silhouette_vals[model.labels_ == c]
            c_silhouette_vals.sort()
            y_upper += len(c_silhouette_vals)
            color = cm.jet(float(i) / n_clusters)
            plt.barh(range(y_lower, y_upper), c_silhouette_vals, height=1.0,
                     edgecolor='none', color=color)
            yticks.append((y_lower + y_upper) / 2.)
            y_lower += len(c_silhouette_vals)

    silhouette_avg = np.mean(silhouette_vals)
    if plot:
        plt.axvline(silhouette_avg, color="red", linestyle="--")
        plt.show()
    print('{} Clusters: Average silhouette coefficient: {:.3f}'.format(n, silhouette_avg))

for n in range(20, 25):
    silhouette(vector, n, plot=False)

### 非階層的クラスタリング

In [ ]:
# KMeans の初期化
# - n_clusters=22: クラスタ数は 22
clusters = KMeans(n_clusters=22).fit_predict(vector)

# 結果を DataFrame にまとめる
df_cluster = pd.DataFrame(clusters, columns=['cluster'])
df_cluster['text'] = df.text

# 確認
df_cluster

In [ ]:
# クラスタ 0 のテキスト
df_cluster.query('cluster==0')

In [ ]:
# クラスタ 1 のテキスト
df_cluster.query('cluster==1')

In [ ]:
# クラスタ 2 のテキスト
df_cluster.query('cluster==2')